# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 4:21PM,264706,15,WT PNC459027B(7994963) Out,"Person & Covey, Inc.",Released
1,Jun 13 2023 4:21PM,264706,15,WT PNC459038B(7994965) Out,"Person & Covey, Inc.",Released
2,Jun 13 2023 4:21PM,264706,15,WT PNC459140A(8003262) Out,"Person & Covey, Inc.",Released
3,Jun 13 2023 4:21PM,264706,15,WT PNC459143A(8003265) Out,"Person & Covey, Inc.",Released
4,Jun 13 2023 4:21PM,264706,15,WT PNC459194A(8006142) Out,"Person & Covey, Inc.",Released
5,Jun 13 2023 3:58PM,264705,22,650029,"NBTY Global, Inc.",Released
6,Jun 13 2023 3:55PM,264704,22,650028,"NBTY Global, Inc.",Released
7,Jun 13 2023 3:49PM,264703,19,ADV80021057,"AdvaGen Pharma, Ltd",Released
8,Jun 13 2023 3:48PM,264702,10,Eme-113183,Emerginnova,Released
9,Jun 13 2023 3:47PM,264701,10,SONSB0002228,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,264702,Released,1
16,264703,Released,1
17,264704,Released,1
18,264705,Released,1
19,264706,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264702,NaN,NaN,1.0
264703,NaN,NaN,1.0
264704,NaN,NaN,1.0
264705,NaN,NaN,1.0
264706,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,16.0,5.0,5.0
264664,0.0,0.0,1.0
264674,0.0,0.0,6.0
264682,0.0,1.0,0.0
264683,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,16,5,5
264664,0,0,1
264674,0,0,6
264682,0,1,0
264683,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,16,5,5
1,264664,0,0,1
2,264674,0,0,6
3,264682,0,1,0
4,264683,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,16,5,5
1,264664,,,1
2,264674,,,6
3,264682,,1,
4,264683,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 4:21PM,264706,15,"Person & Covey, Inc."
5,Jun 13 2023 3:58PM,264705,22,"NBTY Global, Inc."
6,Jun 13 2023 3:55PM,264704,22,"NBTY Global, Inc."
7,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd"
8,Jun 13 2023 3:48PM,264702,10,Emerginnova
9,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC"
10,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc."
11,Jun 13 2023 3:04PM,264695,10,"Methapharm, Inc."
15,Jun 13 2023 3:03PM,264694,10,Methapharm-G
16,Jun 13 2023 2:28PM,264687,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2023 4:21PM,264706,15,"Person & Covey, Inc.",,,5
1,Jun 13 2023 3:58PM,264705,22,"NBTY Global, Inc.",,,1
2,Jun 13 2023 3:55PM,264704,22,"NBTY Global, Inc.",,,1
3,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",,,1
4,Jun 13 2023 3:48PM,264702,10,Emerginnova,,,1
5,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC",,,1
6,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc.",,,1
7,Jun 13 2023 3:04PM,264695,10,"Methapharm, Inc.",,,4
8,Jun 13 2023 3:03PM,264694,10,Methapharm-G,,,1
9,Jun 13 2023 2:28PM,264687,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 4:21PM,264706,15,"Person & Covey, Inc.",5,,
1,Jun 13 2023 3:58PM,264705,22,"NBTY Global, Inc.",1,,
2,Jun 13 2023 3:55PM,264704,22,"NBTY Global, Inc.",1,,
3,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1,,
4,Jun 13 2023 3:48PM,264702,10,Emerginnova,1,,
5,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC",1,,
6,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc.",1,,
7,Jun 13 2023 3:04PM,264695,10,"Methapharm, Inc.",4,,
8,Jun 13 2023 3:03PM,264694,10,Methapharm-G,1,,
9,Jun 13 2023 2:28PM,264687,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 4:21PM,264706,15,"Person & Covey, Inc.",5,,
1,Jun 13 2023 3:58PM,264705,22,"NBTY Global, Inc.",1,,
2,Jun 13 2023 3:55PM,264704,22,"NBTY Global, Inc.",1,,
3,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1,,
4,Jun 13 2023 3:48PM,264702,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 4:21PM,264706,15,"Person & Covey, Inc.",5.0,NaN,NaN
1,Jun 13 2023 3:58PM,264705,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 13 2023 3:55PM,264704,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
4,Jun 13 2023 3:48PM,264702,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 4:21PM,264706,15,"Person & Covey, Inc.",5.0,0.0,0.0
1,Jun 13 2023 3:58PM,264705,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 13 2023 3:55PM,264704,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
4,Jun 13 2023 3:48PM,264702,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2382220,27.0,0.0,0.0
15,264706,5.0,0.0,0.0
19,1058725,7.0,6.0,16.0
20,529349,4.0,0.0,0.0
22,529409,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2382220,27.0,0.0,0.0
1,15,264706,5.0,0.0,0.0
2,19,1058725,7.0,6.0,16.0
3,20,529349,4.0,0.0,0.0
4,22,529409,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,27.0,0.0,0.0
1,15,5.0,0.0,0.0
2,19,7.0,6.0,16.0
3,20,4.0,0.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,27.0
1,15,Released,5.0
2,19,Released,7.0
3,20,Released,4.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,22
Status,,,,,
Completed,0.0,0.0,16.0,0.0,0.0
Executing,0.0,0.0,6.0,0.0,0.0
Released,27.0,5.0,7.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,22
0,Completed,0.0,0.0,16.0,0.0,0.0
1,Executing,0.0,0.0,6.0,0.0,0.0
2,Released,27.0,5.0,7.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,22
0,Completed,0.0,0.0,16.0,0.0,0.0
1,Executing,0.0,0.0,6.0,0.0,0.0
2,Released,27.0,5.0,7.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()